In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import scipy as sp

In [2]:
def sigmoid(x):
    z = np.exp(-x)

    return 1 / (1 + z)

In [3]:
# Loading Iris Datasets:
iris = sns.load_dataset("iris")

In [4]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
X = iris.copy().iloc[:, :-2].values
y = pd.get_dummies(iris.copy().iloc[:, -1]).values

In [6]:
k = 5

In [7]:
hyper_edges_list = []
n = X.shape[0]
m = X.shape[1]
for i in range(n):
    dist = np.linalg.norm(X[i] - X, axis=1)
    hyper_edge = np.zeros(n)
    hyper_edge[np.argsort(dist)[:k + 1]] = 1
    hyper_edges_list.append(hyper_edge.reshape(-1, 1))
    
H_preprocess = np.hstack(hyper_edges_list)
while True:
    shape_pre = H_preprocess.shape
    unique = np.array(list(set(tuple(h.flatten()) for h in H_preprocess)))
    unique2  = np.array(list(set(tuple(h.flatten()) for h in H_preprocess.T)))
    mask1 = np.sort([np.argmax(np.sum(H_preprocess == u, axis=1)) for u in unique])
    mask2 = np.sort([np.argmax(np.sum(H_preprocess.T == u, axis=1)) for u in unique2])
    
    mask = list(set(mask1) & set(mask2))
    
    H_preprocess = H_preprocess[mask]
    H_preprocess = H_preprocess[:, mask]
    
    if H_preprocess.shape == shape_pre:
        H = H_preprocess
        break

In [8]:
size = H.shape[0]
D_v = np.eye(size) * H.sum(axis= 1)
D_e = np.eye(size) * H.sum(axis= 0)

In [9]:
D_v_inv = np.linalg.inv(D_v)
D_e_inv = np.linalg.inv(D_e)
D_v_inv_frac = sp.linalg.fractional_matrix_power(D_v, -0.5)
D_e_inv_frac = sp.linalg.fractional_matrix_power(D_e, -0.5)
H_inv = np.linalg.inv(H)

In [10]:
S = D_v_inv_frac @ H @ D_e_inv @ H_inv @ D_v_inv_frac

In [11]:
X = X[mask]
y = y[mask]

In [12]:
L = 50
Theta = np.random.normal(size=(m, L))

In [13]:
Z = sigmoid(S @ X @ Theta)

C:\Users\ideapad 330\.conda\envs\AIML\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [14]:
I = np.eye(L, L)

In [15]:
Z.shape

(119, 50)

In [16]:
beta = np.linalg.inv(Z.T @ S.T @ S @ Z + I) @ Z.T @ S.T @ y

In [17]:
beta

array([[ 7.76936064e-14,  1.63868685e-13, -8.72057598e-13],
       [ 1.00822158e-13,  6.09966287e-13, -6.99116566e-13],
       [-6.03557332e-14,  5.76992914e-14,  8.83658657e-13],
       [ 5.51063444e-14, -3.51807841e-13, -1.12443697e-12],
       [-1.32071407e-13, -5.17519597e-13,  1.22673729e-12],
       [-2.79416662e-14, -5.21279625e-14,  4.68953240e-13],
       [-2.99984519e-14, -5.25269354e-14,  3.58590706e-13],
       [ 7.90905231e-15,  2.88826843e-14, -1.44877608e-13],
       [-5.00698203e-15, -4.48513138e-13, -3.43705896e-13],
       [-6.26490241e-15, -5.29868514e-13, -4.66198501e-13],
       [-5.69383356e-15,  3.25538096e-13,  4.33048843e-13],
       [ 2.44671484e-15, -5.65246299e-14, -1.29797814e-13],
       [ 1.13517675e-15,  1.92617324e-13,  2.13530127e-13],
       [ 3.87511390e-15, -6.84861155e-14, -1.30576797e-13],
       [ 2.17936109e-15,  1.31980511e-14, -1.00172911e-14],
       [ 7.29628124e-15,  2.62527780e-13,  1.59386686e-13],
       [-2.67764288e-15, -1.56512842e-13

In [18]:
np.argmax(S @ Z @ beta, axis=1)

array([2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2, 0, 2,
       1, 0, 2, 0, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 1,
       0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 0, 1, 0, 1,
       0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0,
       0, 2, 2, 2, 2, 2, 0, 0, 0], dtype=int64)

In [19]:
np.argmax(y, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)